In [ ]:
import yaml
from pynanomapper import aa
from pynanomapper import units
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr

import io
import numpy as np
import pandas as pd
import qgrid

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from tdigest import TDigest

import warnings
warnings.simplefilter("ignore")

In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))

# Endpoints

In [ ]:
query_widget=widgets.Text(
        placeholder='',
        value="owner_name_s:GRACIOUS",
        description="Query",
        disabled=False,
        style=style
)
display(query_widget)

In [ ]:
reload(client_solr)

facets = client_solr.Facets()

q=query_widget.value
df = facets.summary(service_uri,auth_object, query=q,fields=["topcategory_s","endpointcategory_s","effectendpoint_s","unit_s"])    
qgrid_widget = qgrid.show_grid(df)
qgrid_widget

In [ ]:
df_filtered = qgrid_widget.get_changed_df()
df_filtered["effectendpoint_s"].values


In [ ]:
import math

def endpoint2digest(endpoint,maxrows=10000,q=query_widget.value, debug=False):
        
    _fl = "s_uuid_s,effectendpoint_s,loValue_d,unit_s"
    _query = {"fl":_fl,"fq":"type_s:study","wt":"csv", "rows":maxrows, "q":'effectendpoint_s:"{}" AND {}'.format(endpoint,q) }
    r = client_solr.get(service_uri,query=_query,auth=auth_object)
    df=pd.read_csv(io.StringIO(r.content.decode('utf-8')))
    df = df[df["loValue_d"].notna()]
    if df.shape[0]<=1:
        return ({ "endpoint" : endpoint, "digest" : None})
    try:
        digest = TDigest()
        digest.batch_update(df["loValue_d"])
        df["CDF"] = df["loValue_d"].apply(lambda x: digest.cdf(x))
        df["PQ"] = df["loValue_d"].apply(lambda x: math.ceil(20*digest.cdf(x)))
        if debug:
            display(df.head())    
        return ({ "endpoint" : endpoint, "digest" : digest})
    except Exception as err:
        return ({ "endpoint" : endpoint, "digest" : None})

map_iterator = map(endpoint2digest,df_filtered["effectendpoint_s"].values)
for m in map_iterator:
    if m["digest"] is None:
        continue
    else:        
        print("{} {}".format(m["endpoint"],m["digest"]))
    

In [ ]:
def plot_cdf(m):
    _x_label="m"
    tmp = pd.DataFrame(digest.centroids_to_list())   
    tmp['CDF'] = tmp[_x_label].apply(lambda x: digest.cdf(x))
    #tmp['PQ'] = tmp[_x_label].apply(lambda x: 'C ' * math.ceil(20*digest.cdf(x)))
    tmp['PQ'] = tmp[_x_label].apply(lambda x:  math.ceil(20*digest.cdf(x)))
    #display(tmp)
    ax = sns.relplot(x=_x_label, y="CDF", data=tmp,kind="line")
    plt.title("Cumulative distribution function {}".format(m["endpoint"]))
    display(ax)    

map_iterator = map(endpoint2digest,df_filtered["effectendpoint_s"].values)
for m in map_iterator:
    digest = m["digest"]
    if digest is None:
        continue
    plot_cdf(m)   
    #print(digest.to_dict())

    #digest.to_dict()